In [1]:
# import libraries
import requests
import pandas as pd
from datetime import datetime
import folium
import ipywidgets as widgets
from IPython.display import display
import io

In [2]:
# Import station data with longitude and latitude (for the use later)
df_station = pd.read_csv('taipei_metro_station.csv')
df_station

,station,lat,lon
0,松山機場,25.063000,121.551996
1,中山國中,25.060849,121.544227
2,南京復興,25.052319,121.544011
3,忠孝復興,25.041629,121.543767
4,大安,25.032943,121.543551
...,...,...,...
117,幸福,25.050282,121.460216
118,新北產業園區,25.061548,121.459926
119,G大坪林,24.982899,121.541352
120,O景安,24.993905,121.505113


In [3]:
# Dropdown for Year
dropdown1 = widgets.Dropdown(
    options=['2017', '2018', '2019', '2020', '2021', '2022', '2023', 'All'],
    description='Year:',
    style={'description_width': 'initial'},  # This line allows the full description to be shown
    layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)

# Dropdown for Month
dropdown2 = widgets.Dropdown(
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 'All'],
    description='Month:',
    style={'description_width': 'initial'},  # This line allows the full description to be shown
    layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)

# Dropdown for Day
dropdown3 = widgets.Dropdown(
    options=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', 'All'],
    description='Day:',
    style={'description_width': 'initial'},  # This line allows the full description to be shown
    layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)

# Dropdown for Hour
dropdown4 = widgets.Dropdown(
    options=['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
             '16', '17', '18', '19', '20', '21', '22', '23', 'All'],
    description='Hour:',
    style={'description_width': 'initial'},  # This line allows the full description to be shown
    layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)

# Dropdown for Origin           
dropdown5 = widgets.Dropdown(
   options=['松山機場', '中山國中', '南京復興', '忠孝復興', '大安', '科技大樓', '六張犁', '麟光', '辛亥',\
       '萬芳醫院', '萬芳社區', '木柵', '動物園', '大直', '劍南路', '西湖', '港墘', '文德', '內湖',\
       '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '小碧潭', '新店', '新店區公所', '七張',\
       '大坪林', '景美', '萬隆', '公館', '台電大樓', '古亭', '中正紀念堂', '小南門', '頂溪',\
       '永安市場', '景安', '南勢角', '台大醫院', '台北車站', '中山', '雙連', '民權西路', '圓山',\
       '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投', '新北投', '復興崗',\
       '忠義', '關渡', '竹圍', '紅樹林', '淡水', '頂埔', '永寧', '土城', '海山', '亞東醫院',\
       '府中', 'BL板橋', '新埔', '江子翠', '龍山寺', '西門', '善導寺', '忠孝新生', '忠孝敦化',\
       '國父紀念館', '市政府', '永春', '後山埤', '昆陽', '南港', '象山', '台北101/世貿', '信義安和',\
       '大安森林公園', '北門', '松江南京', '台北小巨蛋', '南京三民', '松山', '輔大', '新莊', '頭前庄',\
       '先嗇宮', '三重', '菜寮', '台北橋', '大橋頭站', '中山國小', '行天宮', '東門', '蘆洲',\
       '三民高中', '徐匯中學', '三和國中', '三重國小', '迴龍', '丹鳳', '十四張', '秀朗橋', '景平',\
       '中和', '橋和', '中原', '板新', 'Y板橋', '新埔民生', '幸福', '新北產業園區', 'All'],
   description='Origin:',
   style={'description_width': 'initial'},  # This line allows the full description to be shown
   layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)

# Dropdown for Destination
dropdown6 = widgets.Dropdown(
   options=['松山機場', '中山國中', '南京復興', '忠孝復興', '大安', '科技大樓', '六張犁', '麟光', '辛亥',\
       '萬芳醫院', '萬芳社區', '木柵', '動物園', '大直', '劍南路', '西湖', '港墘', '文德', '內湖',\
       '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館', '小碧潭', '新店', '新店區公所', '七張',\
       'G大坪林', '景美', '萬隆', '公館', '台電大樓', '古亭', '中正紀念堂', '小南門', '頂溪',\
       '永安市場', 'O景安', '南勢角', '台大醫院', '台北車站', '中山', '雙連', '民權西路', '圓山',\
       '劍潭', '士林', '芝山', '明德', '石牌', '唭哩岸', '奇岩', '北投', '新北投', '復興崗',\
       '忠義', '關渡', '竹圍', '紅樹林', '淡水', '頂埔', '永寧', '土城', '海山', '亞東醫院',\
       '府中', 'BL板橋', '新埔', '江子翠', '龍山寺', '西門', '善導寺', '忠孝新生', '忠孝敦化',\
       '國父紀念館', '市政府', '永春', '後山埤', '昆陽', '南港', '象山', '台北101/世貿', '信義安和',\
       '大安森林公園', '北門', '松江南京', '台北小巨蛋', '南京三民', '松山', '輔大', '新莊', 'O頭前庄',\
       '先嗇宮', '三重', '菜寮', '台北橋', '大橋頭站', '中山國小', '行天宮', '東門', '蘆洲',\
       '三民高中', '徐匯中學', '三和國中', '三重國小', '迴龍', '丹鳳', 'All'],
   description='Destination:',
   style={'description_width': 'initial'},  # This line allows the full description to be shown
   layout=widgets.Layout(width='20%', align_self='flex-end') # 'center', 'flex-start', 'flex-end', or 'stretch'
)


# Define a function to update the options of the dependent dropdowns
def update_options1(change):
    if change['new'] == 'All':
        dropdown2.options = ['All']
        dropdown3.options = ['All']
        dropdown4.options = ['All']
    else:
        # Update options based on the selected year
        # Add your logic to update options based on the selected year
        # For example, you can update months, days, and hours accordingly
        dropdown2.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 'All']
        dropdown3.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',\
                             '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', \
                             '31', 'All']
        dropdown4.options = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', \
                             '15', '16', '17', '18', '19', '20', '21', '22', '23', 'All']
    # Reset the month dropdown when the year is changed
        reset_month_dropdown()
    
def update_options2(change):
    if change['new'] == 'All':
        dropdown3.options = ['All']
        dropdown4.options = ['All']
    else:
        # Update options based on the selected month
        if change['new'] in ['01', '03', '05', '07', '08', '10', '12']:
            # For example, you can update months, days, and hours accordingly
            dropdown3.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',\
                                 '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',\
                                 '31', 'All']
        elif change['new'] in ['04', '06', '09', '11']:
            dropdown3.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',\
                                 '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',\
                                 'All']
        elif change['new'] == '02' and int(dropdown1.value)%4 == 0:
            dropdown3.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',\
                                 '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', 'All']
        elif change['new'] == '02' and int(dropdown1.value)%4 !=0:
            dropdown3.options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',\
                                 '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', 'All']
        
        dropdown4.options = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',\
                             '15', '16', '17', '18', '19', '20', '21', '22', '23', 'All']
        # Reset the day dropdown when the month is changed
        reset_day_dropdown()

def update_options3(change):
    if change['new'] == 'All':
        dropdown4.options = ['All']
    else:
        # Update options based on the selected date
        # For example, you can update months, days, and hours accordingly
        dropdown4.options = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14',\
                             '15', '16', '17', '18', '19', '20', '21', '22', '23', 'All']
        # Reset the hour dropdown when the day is changed
        reset_hour_dropdown()     

def reset_month_dropdown():
    dropdown2.value = 'All'  # You can set it to the default value you prefer
    update_options2({'new': 'All'})  # Call the update_options2 function to reset the day and hour dropdowns

def reset_day_dropdown():
    dropdown3.value = 'All'  # You can set it to the default value you prefer
    update_options3({'new': 'All'})  # Call the update_options3 function to reset the hour dropdown

def reset_hour_dropdown():
    dropdown4.value = 'All'  # You can set it to the default value you prefer
    

# Attach the update_options function to the 'value' attribute of the Year, Month, Day dropdowns
dropdown1.observe(update_options1, names='value')
dropdown2.observe(update_options2, names='value')
dropdown3.observe(update_options3, names='value')

# Display the dropdowns
#widgets.VBox([dropdown1, dropdown2, dropdown3, dropdown4])
display(dropdown1, dropdown2, dropdown3, dropdown4, dropdown5, dropdown6)

Dropdown(description='Year:', layout=Layout(align_self='flex-end', width='20%'), options=('2017', '2018', '201…

Dropdown(description='Month:', layout=Layout(align_self='flex-end', width='20%'), options=('01', '02', '03', '…

Dropdown(description='Day:', layout=Layout(align_self='flex-end', width='20%'), options=('01', '02', '03', '04…

Dropdown(description='Hour:', layout=Layout(align_self='flex-end', width='20%'), options=('00', '01', '02', '0…

Dropdown(description='Origin:', layout=Layout(align_self='flex-end', width='20%'), options=('松山機場', '中山國中', '南…

Dropdown(description='Destination:', layout=Layout(align_self='flex-end', width='20%'), options=('松山機場', '中山國中…

In [4]:
if dropdown1.value == 'All' and dropdown2.value == 'All':
    URL = "https://data.taipei/api/v1/dataset/eb481f58-1238-4cff-8caa-fa7bb20cb4f4?scope=resourceAquire&limit=1000"
elif dropdown1.value != 'All' and dropdown2.value == 'All':
    URL_original = "https://data.taipei/api/v1/dataset/eb481f58-1238-4cff-8caa-fa7bb20cb4f4?scope=resourceAquire"
    if dropdown1.value == '2017':
        URL = URL_original + '&limit=12'
    elif dropdown1.value != '2017':
        id_num = 12 * (int(dropdown1.value) - 2017)
        add_part = '&offset=' + str(id_num) + '&limit=12'
        URL = URL_original + add_part
else:
    URL_original = "https://data.taipei/api/v1/dataset/eb481f58-1238-4cff-8caa-fa7bb20cb4f4?scope=resourceAquire"
    URL = URL_original + '&q=年月 ' + dropdown1.value + dropdown2.value

# Calling URL 的資料 
response = requests.get(URL)


In [5]:
datelist = []
df_list = []

for i in range(len(response.json()['result']['results'])):
    datelist.append(response.json()['result']['results'][i]['年月'])  
    
    res = requests.get(response.json()['result']['results'][i]['url'])
    res_cont = res.content
    df_list.append(pd.read_csv(io.BytesIO(res_cont)))

df_all = pd.concat(df_list)
    
    #csv_file = open(response.json()['result']['results'][i]['年月'] + '.csv', 'wb')
    #csv_file.write(res_cont)
    #csv_file.close()
    
#testlist = ['201901', '201902'] #delete

#df_list = []

#for i in testlist: #datelist
#    df_i = pd.read_csv(i + ".csv")
#    df_list.append(df_i)
    
#df_all = pd.concat(df_list)

In [6]:
df_all['date'] = pd.to_datetime(df_all['日期'])
df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day'] = df_all['date'].dt.day

df_all.drop(columns = ['日期', 'date'], inplace=True)

In [9]:
# Case 1: Specific OD pair
if dropdown5.value != 'All' and dropdown6.value != 'All':
    if dropdown1.value != 'All':
        if dropdown2.value != 'All':
            if dropdown3.value != 'All':
                if dropdown4.value != 'All':
                    df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                                & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))\
                                & (df_all['時段'] == int(dropdown4.value)) & (df_all['進站'] == dropdown5.value)]
                    df = df.reset_index(drop=True)
                    
                else:
                    df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                                & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))\
                                & (df_all['進站'] == dropdown5.value)]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day']).sum().reset_index().drop(columns=['時段'])
            
            else:
                df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                            & (df_all['month'] == int(dropdown2.value)) & (df_all['進站'] == dropdown5.value)]
                df = df.groupby(['進站', '出站', 'year', 'month']).sum().reset_index().drop(columns=['時段', 'day'])
        
        else:
            df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                        & (df_all['進站'] == dropdown5.value)]
            df = df.groupby(['進站', '出站', 'year']).sum().reset_index().drop(columns=['時段', 'day', 'month'])
    
    else:
        df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['進站'] == dropdown5.value)]
        df = df.groupby(['進站', '出站']).sum().reset_index().drop(columns=['時段', 'day', 'month', 'year'])
        
    print('The tap-in/-out record for this specific OD pair is: ')
    print(df)

# Case 2: Origin is "All"
elif dropdown5.value == 'All' and dropdown6.value != 'All':
    if dropdown1.value != 'All':
        if dropdown2.value != 'All':
            if dropdown3.value != 'All':
                if dropdown4.value != 'All':
                    df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                                & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))\
                                & (df_all['時段'] == int(dropdown4.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day', '時段']).sum().\
                    sort_values(by =['人次'], ascending=False).reset_index()
                    
                else:
                    df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                        & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day']).sum().\
                    sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段'])
                    
            else:
                df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))\
                    & (df_all['month'] == int(dropdown2.value))]
                df = df.groupby(['進站', '出站', 'year', 'month']).sum().\
                sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day'])
        
        else:
            df = df_all[(df_all['出站'] == dropdown6.value) & (df_all['year'] == int(dropdown1.value))]
            df = df.groupby(['進站', '出站', 'year']).sum().\
            sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month'])
            
    else:
        df = df_all[(df_all['出站'] == dropdown6.value)]
        df = df.groupby(['進站', '出站']).sum().\
        sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month', 'year'])

    print('The top 10 OD pairs with the highest tap-in/-out record are:')
    print(df.head(10))
    
    df_merge = pd.merge(df, df_station, left_on = '進站', right_on = 'station')
    df_merge.drop(columns = ['station'], inplace=True)
        
    # Assuming df_merge has station information with latitude ('lat'), longitude ('lon'), and people volume ('人次')
    # Choose a center point based on the coordinates (Avoid zoom in/out)
    center = [sum(df_merge['lat']) / len(df_merge), sum(df_merge['lon']) / len(df_merge)]

    # Create a Folium map centered at the chosen center with a fixed zoom level and disable dragging and zooming
    mymap = folium.Map(location=center, zoom_start=11.5, dragging=True, scrollWheelZoom=True)

    max_volume = max(df_merge['人次'])

    # Add CircleMarkers to the map with radius based on the '人次' column
    for _, row in df_merge.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=row['人次'] /max_volume*20,  # Adjust the scaling factor as needed
            color='red',
            fill=True,
            fill_color='orange',
            fill_opacity=0.6,
            popup='車站: {}<br>人次: {}'.format(row['進站'], row['人次'])
        ).add_to(mymap)

    # Display the map
    display(mymap)
    
# Case 3: Destination is "All"
elif dropdown5.value != 'All' and dropdown6.value == 'All':
    if dropdown1.value != 'All':
        if dropdown2.value != 'All':
            if dropdown3.value != 'All':
                if dropdown4.value != 'All':
                    df = df_all[(df_all['進站'] == dropdown5.value) & (df_all['year'] == int(dropdown1.value))\
                                & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))\
                                & (df_all['時段'] == int(dropdown4.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day', '時段']).sum().\
                        sort_values(by =['人次'], ascending=False).reset_index()
                
                else:
                    df = df_all[(df_all['進站'] == dropdown5.value) & (df_all['year'] == int(dropdown1.value))\
                        & (df_all['month'] == int(dropdown2.value)) & (df_all['day'] == int(dropdown3.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day']).sum().\
                        sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段'])
            
            else:
                df = df_all[(df_all['進站'] == dropdown5.value) & (df_all['year'] == int(dropdown1.value))\
                    & (df_all['month'] == int(dropdown2.value))]
                df = df.groupby(['進站', '出站', 'year', 'month']).sum().\
                    sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day'])
                
        else:
            df = df_all[(df_all['進站'] == dropdown5.value) & (df_all['year'] == int(dropdown1.value))]
            df = df.groupby(['進站', '出站', 'year']).sum().\
                sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month'])    
            
    else:
        df = df_all[(df_all['進站'] == dropdown5.value)]
        df = df.groupby(['進站', '出站']).sum().\
            sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month', 'year'])  

    print('The top 10 OD pairs with the highest tap-in/-out record are:')
    print(df.head(10))

    df_merge = pd.merge(df, df_station, left_on = '出站', right_on = 'station')
    df_merge.drop(columns = ['station'], inplace=True)
    
    # Assuming df_merge has station information with latitude ('lat'), longitude ('lon'), and people volume ('人次')
    # Choose a center point based on the coordinates (Avoid zoom in/out)
    center = [sum(df_merge['lat']) / len(df_merge), sum(df_merge['lon']) / len(df_merge)]

    # Create a Folium map centered at the chosen center with a fixed zoom level and disable dragging and zooming
    mymap = folium.Map(location=center, zoom_start=11.5, dragging=True, scrollWheelZoom=True)

    max_volume = max(df_merge['人次'])

    # Add CircleMarkers to the map with radius based on the '人次' column
    for _, row in df_merge.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=row['人次'] /max_volume*20,  # Adjust the scaling factor as needed
            color='red',
            fill=True,
            fill_color='orange',
            fill_opacity=0.6,
            popup='車站: {}<br>人次: {}'.format(row['出站'], row['人次'])
        ).add_to(mymap)

    # Display the map
    display(mymap)

# Case 4: All OD pairs
elif dropdown5.value == 'All' and dropdown6.value == 'All':
    if dropdown1.value != 'All':
        if dropdown2.value != 'All':
            if dropdown3.value != 'All':
                if dropdown4.value != 'All':
                    df = df_all[(df_all['year'] == int(dropdown1.value)) & (df_all['month'] == int(dropdown2.value))\
                                & (df_all['day'] == int(dropdown3.value)) & (df_all['時段'] == int(dropdown4.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day', '時段']).sum().\
                        sort_values(by =['人次'], ascending=False).reset_index()
                
                else:
                    df = df_all[(df_all['year'] == int(dropdown1.value)) & (df_all['month'] == int(dropdown2.value))\
                                & (df_all['day'] == int(dropdown3.value))]
                    df = df.groupby(['進站', '出站', 'year', 'month', 'day']).sum().\
                        sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段'])
            
            else:
                df = df_all[(df_all['year'] == int(dropdown1.value)) & (df_all['month'] == int(dropdown2.value))]
                df = df.groupby(['進站', '出站', 'year', 'month']).sum().\
                    sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day'])
        
        else:
            df = df_all[(df_all['year'] == int(dropdown1.value))]
            df = df.groupby(['進站', '出站', 'year']).sum().\
                sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month'])
    
    else:
        df = df_all
        df = df.groupby(['進站', '出站']).sum().\
            sort_values(by =['人次'], ascending=False).reset_index().drop(columns=['時段', 'day', 'month', 'year'])
    
    print('The top 10 OD pairs with the highest tap-in/-out record are:')
    print(df.head(10))
    print('\nThe top 10 OD pairs with the lowest tap-in/-out record are:')
    print(df.tail(10))

The top 10 OD pairs with the highest tap-in/-out record are:
         進站    出站  year  month  day     人次
0        西門  台北車站  2017      1    1  12917
1        淡水  台北車站  2017      1    1  11554
2        劍潭  台北車站  2017      1    1   7140
3     中正紀念堂  台北車站  2017      1    1   7093
4       市政府  台北車站  2017      1    1   6431
5  台北101/世貿  台北車站  2017      1    1   5582
6      忠孝復興  台北車站  2017      1    1   4824
7        圓山  台北車站  2017      1    1   4168
8      忠孝敦化  台北車站  2017      1    1   4120
9        士林  台北車站  2017      1    1   3891
